In [ ]:
from pymilvus import MilvusClient

host = "localhost"
port = "19530"

milvus_client = MilvusClient(
    host=host,
    port=port
)


In [ ]:
from pymilvus import FieldSchema, CollectionSchema, DataType

VECTOR_LENGTH = 768  # Dimensionality for Silver Retriever Base (v1.1)

# Define the fields
id_field = FieldSchema(
    name="id",
    dtype=DataType.INT64,
    is_primary=True,
    description="Primary id"
)
text_field = FieldSchema(
    name="text",
    dtype=DataType.VARCHAR,
    max_length=4096,
    description="Original text content"
)
embedding_field = FieldSchema(
    name="embedding",
    dtype=DataType.FLOAT_VECTOR,
    dim=VECTOR_LENGTH,
    description="Text embeddings"
)

# Create list of fields
fields = [id_field, text_field, embedding_field]

# Define the full collection schema
schema = CollectionSchema(
    fields=fields,
    auto_id=True,                # Let Milvus generate IDs automatically
    enable_dynamic_field=True,   # Allow adding extra fields easily later
    description="RAG Texts Collection"
)
